# Design emotional models for chatbots

### 给聊天机器人定制不同的性格（ES），根据用户的情绪和自身的性格计算response的情绪
### Customize different personalities (ES) for the chatbot, and calculate the emotions of response according to the user's emotions and their own personalities.
**ES = {s1, s2, s3, s4, s5}** <br>
s1, s2, s3, s4, s5 indicate the intensity of angry, sad, happy, neutral, and Frustrated respectively, (si∈[0,1], s1+s2+s3+s4+s5 = 1)

### angry- 0,   sad- 1,   happy- 2,   neutral- 3,   frustrated- 4

In [2]:
import numpy as np
import pandas as pd
import pickle
import os, sys
from keras.models import Model, load_model
import keras.backend as K

Using TensorFlow backend.


In [3]:
# 第二代情绪识别模型
model = load_model('../models/emotion_recognition_model.hdf5')
data_all = pickle.load(open("../models/data_word_id_map.pkl","rb"))
id2word, word2id = data_all[4], data_all[5]

In [4]:
# 去掉一些停用词
def filter_sent(sent):
    return sent.replace("'", ' ').replace('-', ' ').replace('.', ' ').replace('?', ' ').replace(':', ' ').replace(';', ' ').replace('(', ' ').replace(')', ' ').replace('!', ' ').replace('/', ' ')

In [5]:
def get_word_indices(data_x, MaxLen):
    length = len(data_x)
    word_list = []
    for word in data_x:
        if not word in word2id:
            # 遇到词汇表之外的词就把它归为unknown
            word_list.append(word2id['<UNK>'])
        else:
            word_list.append(word2id[word])
    return np.array(word_list + [0]*(MaxLen-length))[:MaxLen]

In [6]:
def get_one_hot(label, wordNum):
    label_arr = [0] * wordNum
    label_arr[label]=1
    return label_arr[:]

In [7]:
def expendToOneHot(x, wordNum, MaxLen):
    t = np.expand_dims(x, axis=2)
    t = t.tolist()
    for i in range(len(t)):
        for j in range(MaxLen):
            if(t[i][j][0] == 0):
                t[i][j:] = [[0] * wordNum] * (MaxLen - j)
                break
            else:
                t[i][j] = get_one_hot(t[i][j][0], wordNum)
    return np.array(t)

In [8]:
def processUserInput(userInputString):
    wordNum = len(id2word)
    MaxLen = 50
    # 转小写
    userInputString = userInputString.lower()
    # 按空格split成一个元素为word的list
    split_list = filter_sent(userInputString).split()
    # 将word按照word2id转为id
    sentence_word_indices = get_word_indices(split_list, MaxLen)
    # 扩展维度到合适的input维度（1×MaxLen×wordNum）
    sentence_word_indices = np.expand_dims(sentence_word_indices, axis=0)
    final_input = expendToOneHot(sentence_word_indices, wordNum, MaxLen)    
    return final_input

### 每5次对话将ES向着原始值恢复（不直接复原，每次恢复一点）
### Restore ES to original value every 5 conversations (not directly, one bit at a time)

In [9]:
def reference(ES):
    ES_tmp = ES
    i = 0
    print('original ES: ')
    print(ES)
    print('**********************************')

    while True:
        # 每进行5次对话就将ES向它的原始值恢复（向聊天机器人的一般情绪恢复）
        if i % 5 == 0 and i != 0:
            ES_tmp = ES_tmp * ES
            ES_tmp = ES_tmp / ES_tmp.sum()
            print('==================================')
            print('Restore ES!')
            print('ES:')
            print(ES_tmp)
            print('==================================')
        sentence = input("Input a sentence: ")
        if sentence == 'exit':
            break
        userInput = processUserInput(sentence)

        E = model.predict(userInput)[0]
        RE = E
        UE = RE * ES_tmp * 100
        UE = UE / UE.sum()
        ES_tmp = UE

        print('RE:')
        print(RE)
        print('ES:')
        print(ES_tmp)
        print('UE:')
        print(UE)

        res = UE.argmax()
        if(res == 0):
            res = 'angry'
        elif(res == 1):
            res = 'sad'
        elif(res == 2):
            res = 'happy'
        elif(res == 3):
            res = 'neutral'
        elif(res == 4):
            res = 'frustrated'

        print(res)
        print('----------------------------------')

        i += 1

In [41]:
# angry: ['I hate you!', 'what are you talking about?']
# neutral: ['excuse me sir' ,'excuse me', 'nice to meet you', 'glad to meet you', 'hello', 'hello sir', 'how are you', 'sorry']
# sad: ['I feel bad', 'it's over', 'i feel worse']
# happy: ['I am excited', 'it's good', 'I am so happy', 'it's wonderful!', 'I am very happy']
# frustrated: ['I can't work out this problem!', 'I can't do it!', 'go away!', 'I don't konw what to do']

## Neutral chatbot

### 1. 自身性格的情绪不易被改变，易被恢复
### It is not easy to change and recover the emotions of one's own personality

In [18]:
# ES = np.array([0.025, 0.025, 0.025, 0.9, 0.025])
ES = np.array([0.005, 0.015, 0.015, 0.96, 0.005])
reference(ES)

original ES: 
[0.005 0.015 0.015 0.96  0.005]
**********************************
Input a sentence: i hate you
RE:
[0.7370559  0.03947276 0.062338   0.03730788 0.1238255 ]
ES:
[0.08848819 0.01421686 0.02245221 0.85997672 0.01486603]
UE:
[0.08848819 0.01421686 0.02245221 0.85997672 0.01486603]
neutral
----------------------------------
Input a sentence: i hate you
RE:
[0.7370559  0.03947276 0.062338   0.03730788 0.1238255 ]
ES:
[0.64507133 0.00555038 0.01384312 0.31732865 0.01820652]
UE:
[0.64507133 0.00555038 0.01384312 0.31732865 0.01820652]
angry
----------------------------------
Input a sentence: what are you talking about?
RE:
[0.60040295 0.03087687 0.06436673 0.11102475 0.19332871]
ES:
[9.06785129e-01 4.01245519e-04 2.08616745e-03 8.24865088e-02
 8.24094917e-03]
UE:
[9.06785129e-01 4.01245519e-04 2.08616745e-03 8.24865088e-02
 8.24094917e-03]
angry
----------------------------------
Input a sentence: excuse me sir
RE:
[0.02938027 0.00483743 0.00323106 0.67571205 0.2868391 ]
ES:
[3

### 2. 维持情绪的能力&恢复自身性格的能力（恢复原状--[对立情绪不显著]）
### Ability to Maintain Emotions & Ability to Restore one's own character (Revert --[Non-significant antagonistic emotions])

In [43]:
# ES = np.array([0.025, 0.025, 0.025, 0.9, 0.025])
ES = np.array([0.005, 0.015, 0.015, 0.96, 0.005])
reference(ES)

original ES: 
[0.005 0.015 0.015 0.96  0.005]
**********************************
Input a sentence: i hate you
RE:
[0.7370559  0.03947276 0.062338   0.03730788 0.1238255 ]
ES:
[0.08848819 0.01421686 0.02245221 0.85997672 0.01486603]
UE:
[0.08848819 0.01421686 0.02245221 0.85997672 0.01486603]
neutral
----------------------------------
Input a sentence: i hate you
RE:
[0.7370559  0.03947276 0.062338   0.03730788 0.1238255 ]
ES:
[0.64507133 0.00555038 0.01384312 0.31732865 0.01820652]
UE:
[0.64507133 0.00555038 0.01384312 0.31732865 0.01820652]
angry
----------------------------------
Input a sentence: what are you talking about?
RE:
[0.60040295 0.03087687 0.06436673 0.11102475 0.19332871]
ES:
[9.06785129e-01 4.01245519e-04 2.08616745e-03 8.24865088e-02
 8.24094917e-03]
UE:
[9.06785129e-01 4.01245519e-04 2.08616745e-03 8.24865088e-02
 8.24094917e-03]
angry
----------------------------------
Input a sentence: i hate you
RE:
[0.7370559  0.03947276 0.062338   0.03730788 0.1238255 ]
ES:
[9.93

### 3. 维持情绪的能力&恢复自身性格的能力（没有恢复原状，只是向着性格靠近--[对立情绪显著]）
### The ability to maintain emotions & the ability to restore one's character (not to return to the original state, but to move closer to the character -[significant antagonism])

In [44]:
# ES = np.array([0.025, 0.025, 0.025, 0.9, 0.025])
ES = np.array([0.005, 0.015, 0.015, 0.96, 0.005])
reference(ES)

original ES: 
[0.005 0.015 0.015 0.96  0.005]
**********************************
Input a sentence: i hate you
RE:
[0.7370559  0.03947276 0.062338   0.03730788 0.1238255 ]
ES:
[0.08848819 0.01421686 0.02245221 0.85997672 0.01486603]
UE:
[0.08848819 0.01421686 0.02245221 0.85997672 0.01486603]
neutral
----------------------------------
Input a sentence: i hate you
RE:
[0.7370559  0.03947276 0.062338   0.03730788 0.1238255 ]
ES:
[0.64507133 0.00555038 0.01384312 0.31732865 0.01820652]
UE:
[0.64507133 0.00555038 0.01384312 0.31732865 0.01820652]
angry
----------------------------------
Input a sentence: what are you talking about?
RE:
[0.60040295 0.03087687 0.06436673 0.11102475 0.19332871]
ES:
[9.06785129e-01 4.01245519e-04 2.08616745e-03 8.24865088e-02
 8.24094917e-03]
UE:
[9.06785129e-01 4.01245519e-04 2.08616745e-03 8.24865088e-02
 8.24094917e-03]
angry
----------------------------------
Input a sentence: what are you talking about?
RE:
[0.60040295 0.03087687 0.06436673 0.11102475 0.19

## Angry chatbot

In [36]:
# ES = np.array([0.9, 0.025, 0.025, 0.025, 0.025])
ES = np.array([0.96, 0.01, 0.01, 0.005, 0.015])
reference(ES)

original ES: 
[0.96  0.01  0.01  0.005 0.015]
**********************************
Input a sentence: hello sir
RE:
[0.02335424 0.00842327 0.01331214 0.81149286 0.14341752]
ES:
[0.77722915 0.00292007 0.00461487 0.14065878 0.07457712]
UE:
[0.77722915 0.00292007 0.00461487 0.14065878 0.07457712]
angry
----------------------------------
Input a sentence: glad to meet you
RE:
[0.02415585 0.22824149 0.27083343 0.39599118 0.08077814]
ES:
[0.22780648 0.0080869  0.01516551 0.67584501 0.07309611]
UE:
[0.22780648 0.0080869  0.01516551 0.67584501 0.07309611]
neutral
----------------------------------
Input a sentence: nice to meet you
RE:
[0.08284835 0.16260889 0.08107949 0.4032923  0.27017096]
ES:
[0.06015816 0.00419151 0.00391934 0.86878364 0.06294736]
UE:
[0.06015816 0.00419151 0.00391934 0.86878364 0.06294736]
neutral
----------------------------------
Input a sentence: i hate you
RE:
[0.7370559  0.03947276 0.062338   0.03730788 0.1238255 ]
ES:
[0.52191227 0.00194747 0.00287586 0.38151777 0.0917

In [55]:
ES = np.array([0.96, 0.01, 0.01, 0.005, 0.015])
reference(ES)

original ES: 
[0.96  0.01  0.01  0.005 0.015]
**********************************
Input a sentence: hello sir
RE:
[0.02335424 0.00842327 0.01331214 0.81149286 0.14341752]
ES:
[0.77722915 0.00292007 0.00461487 0.14065878 0.07457712]
UE:
[0.77722915 0.00292007 0.00461487 0.14065878 0.07457712]
angry
----------------------------------
Input a sentence: excuse me sir
RE:
[0.02938027 0.00483743 0.00323106 0.67571205 0.2868391 ]
ES:
[1.63927393e-01 1.01403779e-04 1.07041491e-04 6.82299726e-01
 1.53564436e-01]
UE:
[1.63927393e-01 1.01403779e-04 1.07041491e-04 6.82299726e-01
 1.53564436e-01]
neutral
----------------------------------
Input a sentence: nice to meet you
RE:
[0.08284835 0.16260889 0.08107949 0.4032923  0.27017096]
ES:
[4.11223463e-02 4.99276277e-05 2.62788081e-05 8.33177681e-01
 1.25623767e-01]
UE:
[4.11223463e-02 4.99276277e-05 2.62788081e-05 8.33177681e-01
 1.25623767e-01]
neutral
----------------------------------
Input a sentence: glad to meet you
RE:
[0.02415585 0.22824149 0.

In [50]:
ES = np.array([0.96, 0.01, 0.01, 0.005, 0.015])
reference(ES)

original ES: 
[0.96  0.01  0.01  0.005 0.015]
**********************************
Input a sentence: hello sir
RE:
[0.02335424 0.00842327 0.01331214 0.81149286 0.14341752]
ES:
[0.77722915 0.00292007 0.00461487 0.14065878 0.07457712]
UE:
[0.77722915 0.00292007 0.00461487 0.14065878 0.07457712]
angry
----------------------------------
Input a sentence: excuse me sir
RE:
[0.02938027 0.00483743 0.00323106 0.67571205 0.2868391 ]
ES:
[1.63927393e-01 1.01403779e-04 1.07041491e-04 6.82299726e-01
 1.53564436e-01]
UE:
[1.63927393e-01 1.01403779e-04 1.07041491e-04 6.82299726e-01
 1.53564436e-01]
neutral
----------------------------------
Input a sentence: how are you?
RE:
[0.08277354 0.16152053 0.13788071 0.3707883  0.24703689]
ES:
[4.45574508e-02 5.37847497e-05 4.84655297e-05 8.30765627e-01
 1.24574672e-01]
UE:
[4.45574508e-02 5.37847497e-05 4.84655297e-05 8.30765627e-01
 1.24574672e-01]
neutral
----------------------------------
Input a sentence: nice to meet you
RE:
[0.08284835 0.16260889 0.0810

## Happy chatbot

In [53]:
ES = np.array([0.025, 0.025, 0.9, 0.025, 0.025])
reference(ES)

original ES: 
[0.025 0.025 0.9   0.025 0.025]
**********************************
Input a sentence: i feel bad
RE:
[0.01952218 0.5557889  0.1950897  0.17796235 0.05163694]
ES:
[0.00249385 0.07099885 0.89717731 0.02273367 0.00659632]
UE:
[0.00249385 0.07099885 0.89717731 0.02273367 0.00659632]
happy
----------------------------------
Input a sentence: i feel worse
RE:
[0.00561836 0.9022032  0.05028597 0.02664325 0.01524924]
ES:
[1.27502001e-04 5.82898700e-01 4.10546638e-01 5.51180915e-03
 9.15350631e-04]
UE:
[1.27502001e-04 5.82898700e-01 4.10546638e-01 5.51180915e-03
 9.15350631e-04]
sad
----------------------------------
Input a sentence: it's over
RE:
[0.08334704 0.5502289  0.1293722  0.14117807 0.09587376]
ES:
[2.83597974e-05 8.55918545e-01 1.41742276e-01 2.07662163e-03
 2.34198007e-04]
UE:
[2.83597974e-05 8.55918545e-01 1.41742276e-01 2.07662163e-03
 2.34198007e-04]
sad
----------------------------------
Input a sentence: i am excited
RE:
[0.08154178 0.03088914 0.70402324 0.10111249

In [58]:
ES = np.array([0.025, 0.025, 0.9, 0.025, 0.025])
reference(ES)

original ES: 
[0.025 0.025 0.9   0.025 0.025]
**********************************
Input a sentence: i feel bad
RE:
[0.01952218 0.5557889  0.1950897  0.17796235 0.05163694]
ES:
[0.00249385 0.07099885 0.89717731 0.02273367 0.00659632]
UE:
[0.00249385 0.07099885 0.89717731 0.02273367 0.00659632]
happy
----------------------------------
Input a sentence: i feel worse
RE:
[0.00561836 0.9022032  0.05028597 0.02664325 0.01524924]
ES:
[1.27502001e-04 5.82898700e-01 4.10546638e-01 5.51180915e-03
 9.15350631e-04]
UE:
[1.27502001e-04 5.82898700e-01 4.10546638e-01 5.51180915e-03
 9.15350631e-04]
sad
----------------------------------
Input a sentence: it's over
RE:
[0.08334704 0.5502289  0.1293722  0.14117807 0.09587376]
ES:
[2.83597974e-05 8.55918545e-01 1.41742276e-01 2.07662163e-03
 2.34198007e-04]
UE:
[2.83597974e-05 8.55918545e-01 1.41742276e-01 2.07662163e-03
 2.34198007e-04]
sad
----------------------------------
Input a sentence: i feel worse
RE:
[0.00561836 0.9022032  0.05028597 0.02664325

In [59]:
ES = np.array([0.025, 0.025, 0.9, 0.025, 0.025])
reference(ES)

original ES: 
[0.025 0.025 0.9   0.025 0.025]
**********************************
Input a sentence: i feel bad
RE:
[0.01952218 0.5557889  0.1950897  0.17796235 0.05163694]
ES:
[0.00249385 0.07099885 0.89717731 0.02273367 0.00659632]
UE:
[0.00249385 0.07099885 0.89717731 0.02273367 0.00659632]
happy
----------------------------------
Input a sentence: i feel worse
RE:
[0.00561836 0.9022032  0.05028597 0.02664325 0.01524924]
ES:
[1.27502001e-04 5.82898700e-01 4.10546638e-01 5.51180915e-03
 9.15350631e-04]
UE:
[1.27502001e-04 5.82898700e-01 4.10546638e-01 5.51180915e-03
 9.15350631e-04]
sad
----------------------------------
Input a sentence: it's over
RE:
[0.08334704 0.5502289  0.1293722  0.14117807 0.09587376]
ES:
[2.83597974e-05 8.55918545e-01 1.41742276e-01 2.07662163e-03
 2.34198007e-04]
UE:
[2.83597974e-05 8.55918545e-01 1.41742276e-01 2.07662163e-03
 2.34198007e-04]
sad
----------------------------------
Input a sentence: i feel worse
RE:
[0.00561836 0.9022032  0.05028597 0.02664325

## Sad chatbot

In [69]:
ES = np.array([0.025, 0.92, 0.005, 0.025, 0.025])
reference(ES)

original ES: 
[0.025 0.92  0.005 0.025 0.025]
**********************************
Input a sentence: i am so happy
RE:
[0.00691209 0.06654879 0.6301262  0.25250056 0.04391242]
ES:
[0.00240141 0.85083436 0.04378391 0.08772419 0.01525613]
UE:
[0.00240141 0.85083436 0.04378391 0.08772419 0.01525613]
sad
----------------------------------
Input a sentence: i am excited
RE:
[0.08154178 0.03088914 0.70402324 0.10111249 0.08243331]
ES:
[0.00290399 0.38976112 0.45713997 0.13154424 0.01865069]
UE:
[0.00290399 0.38976112 0.45713997 0.13154424 0.01865069]
happy
----------------------------------
Input a sentence: it's good!
RE:
[0.10976945 0.08740561 0.50043637 0.2133291  0.0890594 ]
ES:
[0.0010884  0.1163188  0.78110633 0.09581512 0.00567135]
UE:
[0.0010884  0.1163188  0.78110633 0.09581512 0.00567135]
happy
----------------------------------
Input a sentence: i feel worse
RE:
[0.00561836 0.9022032  0.05028597 0.02664325 0.01524924]
ES:
[4.16363589e-05 7.14544288e-01 2.67443364e-01 1.73818546e-02


In [70]:
ES = np.array([0.025, 0.92, 0.005, 0.025, 0.025])
reference(ES)

original ES: 
[0.025 0.92  0.005 0.025 0.025]
**********************************
Input a sentence: i am so happy
RE:
[0.00691209 0.06654879 0.6301262  0.25250056 0.04391242]
ES:
[0.00240141 0.85083436 0.04378391 0.08772419 0.01525613]
UE:
[0.00240141 0.85083436 0.04378391 0.08772419 0.01525613]
sad
----------------------------------
Input a sentence: i am excited
RE:
[0.08154178 0.03088914 0.70402324 0.10111249 0.08243331]
ES:
[0.00290399 0.38976112 0.45713997 0.13154424 0.01865069]
UE:
[0.00290399 0.38976112 0.45713997 0.13154424 0.01865069]
happy
----------------------------------
Input a sentence: it's good
RE:
[0.10976945 0.08740561 0.50043637 0.2133291  0.0890594 ]
ES:
[0.0010884  0.1163188  0.78110633 0.09581512 0.00567135]
UE:
[0.0010884  0.1163188  0.78110633 0.09581512 0.00567135]
happy
----------------------------------
Input a sentence: it's wonderful
RE:
[0.01916603 0.08073365 0.42130986 0.409426   0.06936441]
ES:
[5.51681036e-05 2.48354734e-02 8.70321530e-01 1.03747451e-01

In [71]:
ES = np.array([0.025, 0.92, 0.005, 0.025, 0.025])
reference(ES)

original ES: 
[0.025 0.92  0.005 0.025 0.025]
**********************************
Input a sentence: i am so happy
RE:
[0.00691209 0.06654879 0.6301262  0.25250056 0.04391242]
ES:
[0.00240141 0.85083436 0.04378391 0.08772419 0.01525613]
UE:
[0.00240141 0.85083436 0.04378391 0.08772419 0.01525613]
sad
----------------------------------
Input a sentence: i am excited
RE:
[0.08154178 0.03088914 0.70402324 0.10111249 0.08243331]
ES:
[0.00290399 0.38976112 0.45713997 0.13154424 0.01865069]
UE:
[0.00290399 0.38976112 0.45713997 0.13154424 0.01865069]
happy
----------------------------------
Input a sentence: it's good
RE:
[0.10976945 0.08740561 0.50043637 0.2133291  0.0890594 ]
ES:
[0.0010884  0.1163188  0.78110633 0.09581512 0.00567135]
UE:
[0.0010884  0.1163188  0.78110633 0.09581512 0.00567135]
happy
----------------------------------
Input a sentence: it's wonderful
RE:
[0.01916603 0.08073365 0.42130986 0.409426   0.06936441]
ES:
[5.51681036e-05 2.48354734e-02 8.70321530e-01 1.03747451e-01

## Frustrated chatbot

In [79]:
ES = np.array([0.1, 0.1, 0.05, 0.05, 0.7])
reference(ES)

original ES: 
[0.1  0.1  0.05 0.05 0.7 ]
**********************************
Input a sentence: hello
RE:
[0.01301106 0.06822079 0.36891136 0.48665288 0.06320392]
ES:
[0.0136751  0.07170256 0.19386972 0.25574507 0.46500754]
UE:
[0.0136751  0.07170256 0.19386972 0.25574507 0.46500754]
frustrated
----------------------------------
Input a sentence: i am excited
RE:
[0.08154178 0.03088914 0.70402324 0.10111249 0.08243331]
ES:
[0.00546587 0.01085649 0.66903041 0.12675379 0.18789344]
UE:
[0.00546587 0.01085649 0.66903041 0.12675379 0.18789344]
happy
----------------------------------
Input a sentence: i am very happy
RE:
[0.05534358 0.04297711 0.44463572 0.33382902 0.12321454]
ES:
[0.00083171 0.00128284 0.81789194 0.11634045 0.06365306]
UE:
[0.00083171 0.00128284 0.81789194 0.11634045 0.06365306]
happy
----------------------------------
Input a sentence: I can't work out this problem!
RE:
[0.16817482 0.08382186 0.03345519 0.19441134 0.5201368 ]
ES:
[0.00167841 0.00129031 0.32834091 0.27140508

In [81]:
# ES = np.array([0.15, 0.15, 0.05, 0.05, 0.6])
ES = np.array([0.1, 0.1, 0.05, 0.05, 0.7])
reference(ES)

original ES: 
[0.1  0.1  0.05 0.05 0.7 ]
**********************************
Input a sentence: hello
RE:
[0.01301106 0.06822079 0.36891136 0.48665288 0.06320392]
ES:
[0.0136751  0.07170256 0.19386972 0.25574507 0.46500754]
UE:
[0.0136751  0.07170256 0.19386972 0.25574507 0.46500754]
frustrated
----------------------------------
Input a sentence: i am excited
RE:
[0.08154178 0.03088914 0.70402324 0.10111249 0.08243331]
ES:
[0.00546587 0.01085649 0.66903041 0.12675379 0.18789344]
UE:
[0.00546587 0.01085649 0.66903041 0.12675379 0.18789344]
happy
----------------------------------
Input a sentence: i am very happy
RE:
[0.05534358 0.04297711 0.44463572 0.33382902 0.12321454]
ES:
[0.00083171 0.00128284 0.81789194 0.11634045 0.06365306]
UE:
[0.00083171 0.00128284 0.81789194 0.11634045 0.06365306]
happy
----------------------------------
Input a sentence: it's good
RE:
[0.10976945 0.08740561 0.50043637 0.2133291  0.0890594 ]
ES:
[2.07494871e-04 2.54838406e-04 9.30246476e-01 5.64071444e-02
 1.2

In [83]:
ES = np.array([0.1, 0.1, 0.05, 0.05, 0.7])
reference(ES)

original ES: 
[0.1  0.1  0.05 0.05 0.7 ]
**********************************
Input a sentence: hello
RE:
[0.01301106 0.06822079 0.36891136 0.48665288 0.06320392]
ES:
[0.0136751  0.07170256 0.19386972 0.25574507 0.46500754]
UE:
[0.0136751  0.07170256 0.19386972 0.25574507 0.46500754]
frustrated
----------------------------------
Input a sentence: i am very happy
RE:
[0.05534358 0.04297711 0.44463572 0.33382902 0.12321454]
ES:
[0.00325223 0.01324207 0.37042317 0.3668725  0.24621003]
UE:
[0.00325223 0.01324207 0.37042317 0.3668725  0.24621003]
happy
----------------------------------
Input a sentence: it's good
RE:
[0.10976945 0.08740561 0.50043637 0.2133291  0.0890594 ]
ES:
[0.00124354 0.00403174 0.64572076 0.27262333 0.07638063]
UE:
[0.00124354 0.00403174 0.64572076 0.27262333 0.07638063]
happy
----------------------------------
Input a sentence: i am excited
RE:
[0.08154178 0.03088914 0.70402324 0.10111249 0.08243331]
ES:
[2.07494871e-04 2.54838406e-04 9.30246476e-01 5.64071444e-02
 1.2

## 结果分析：
### 对于sad的chatbot，需要上文的大量关于happy的对话才能将其情绪变为happy
如果chatbot是在一个sad的情绪下，少量的happy对话不足以改变chatbot的sad情绪，这就是情绪的保持。
<br><br>
但如果happy的对话大大增多，就会影响chatbot的情绪，使其变happy。
<br>
### 但是只需要一句sad的对话，就可以将其性格转为sad，因为它的性格就是sad，很容易变sad
**其他性格同理**
### frustrated的性格相当于是angry和sad的结合版
### 注意：每5次对话后向聊天机器人自身的性格恢复不是直接恢复到原始性格的情绪，而是增大聊天机器人特定性格情绪的比例（一个happy性格的chatbot，如果恢复时sad占比过大，恢复后还是显示sad情绪，但是会拉近happy与sad的差距，即变得没那么sad了）

## Results Analysis:
### For sad chatbot, it takes a lot of happy conversations above to change its mood to Happy.
If chatbot is in a SAD mood, a small amount of happy conversation is not enough to change chatbot's SAD mood, which is the maintenance of the mood.
<br><br>
But if there is a lot more happy conversations, it will affect chatbot's mood and make it happy.
<br>
### A chatbot's mood might change from happy to sad with just one sad word, and since that's its personality, it's easy to become sad.
**The same goes for other personalities.**
### Frustrated's character is a cross between Angry and SAD.
#### Note: After every 5 conversations, the character of the chatbot itself is not directly restored to the original personality mood, but increases the proportion of the specific mood of the chatbot. (If the proportion of sad is too large in the recovery of a happy chatbot, it will still show sad after recovery, but will narrow the gap between happy and SAD. It becomes less sad).